In [1]:
import os
os.chdir('/Users/zoe/Documents/复旦课程/大数据商业分析/pj2')
import json
import gensim
from gensim.models import Word2Vec
import random
import collections

Using TensorFlow backend.


In [2]:
# 得到歌单
with open('hot.json', 'r', encoding='utf8') as file:
    contents = [json.loads(one) for one in file.readlines()]
print(len(contents))

41032


In [3]:
# 找出songs大于1的
contents_new = [one for one in contents if len(one['songs'])>1]
print(len(contents_new))

# 去重，后面的歌单替换前面的
contents_dst = dict()
for one in contents_new:
    contents_dst[one['id']] = one
contents_dst = list(contents_dst.values())
print(len(contents_dst))

26423
26420


In [4]:
# 歌单中歌曲数大于n的；
len(list(filter(lambda x:True if len(x['songs'])>5 else False, contents_dst)))

26411

In [4]:
# 处理songs
songs = []
with open('songs.json', 'r', encoding='utf8') as file:
    for one in file.readlines():
        new_dict = {}
        one_dict = json.loads(one)
        new_dict['id'] = one_dict['id']
        new_dict['album'] = one_dict['album']
        new_dict['company'] = one_dict['album_company']
        new_dict['artist'] = one_dict['artist']
        new_dict['artist_id'] = one_dict['artist_id']
        new_dict['name'] = one_dict['name']
        new_dict['popularity'] = one_dict['popularity']
        songs.append(new_dict)
    
song_dict = {}
for one in songs:
    song_dict[one['id']] = one

In [ ]:
for one in songs:
    if one['artist'] == '林宥嘉' and one['popularity'] == 100:
        print(one['name'])

In [ ]:
for one in contents_dst[:100]:
    print(list(map(lambda x:song_dict[x]['name'], one['songs'])))

In [31]:
# 遍历所有歌单，生成模型 song2vec
playlist_sequence = []
for one in contents_dst:
    song_sequence = list(map(lambda x:str(x), one['songs']))
    for _ in range(len(song_sequence)//10):
        new = list(song_sequence)
        random.shuffle(new)
        playlist_sequence.append(new)

# out 为输出模型
out = 'song2vec.model'
model = Word2Vec(sentences=playlist_sequence, size=100, window=5, min_count=5)
model.save(out)

In [5]:
# 测试模型song2vec
def test(song_id):
    model = gensim.models.Word2Vec.load("song2vec.model")
    result = model.most_similar(str(song_id))
    print("\n与 {} 的 {} 相似的歌曲有：".format(song_dict[song_id]['artist'],song_dict[song_id]['name']))
    result_li = []
    for one in result:
#         print(((song_dict[int(one[0])]['artist'],song_dict[int(one[0])]['name']), round(float(one[1]), 3)))
        print(song_dict[int(one[0])]['artist']+' '+song_dict[int(one[0])]['name'])
        result_li.append(((song_dict[int(one[0])]['artist'],song_dict[int(one[0])]['name'],song_dict[int(one[0])]['popularity']), round(float(one[1]), 3)))
    return result_li

In [8]:
candidate = []
for one in songs:
    if one['name'] == '成全' and one['artist'] == '林宥嘉':
        candidate.append(one)
sorted(candidate, key=lambda x:x['popularity'], reverse=True)

[{'album': '成全',
  'artist': '林宥嘉',
  'artist_id': 3685,
  'company': '华研',
  'id': 562594267,
  'name': '成全',
  'popularity': 100.0},
 {'album': '蒙面唱将猜猜猜 第一期',
  'artist': '林宥嘉',
  'artist_id': 3685,
  'company': '灿星文化',
  'id': 431259309,
  'name': '成全',
  'popularity': 100.0}]

In [9]:
test(562594267)


与 林宥嘉 的 成全 相似的歌曲有：
五月天 后来的我们
薛之谦 哑巴
李荣浩 戒烟
薛之谦 肆无忌惮
赵紫骅 可乐
周兴哲 你，好不好？
田馥甄 你就不要想起我
毛不易 无问
陈奕迅 我们
田馥甄 爱了很久的朋友


[(('五月天', '后来的我们', 100.0), 0.963),
 (('薛之谦', '哑巴', 100.0), 0.962),
 (('李荣浩', '戒烟', 100.0), 0.955),
 (('薛之谦', '肆无忌惮', 100.0), 0.947),
 (('赵紫骅', '可乐', 100.0), 0.945),
 (('周兴哲', '你，好不好？', 100.0), 0.943),
 (('田馥甄', '你就不要想起我', 100.0), 0.941),
 (('毛不易', '无问', 100.0), 0.941),
 (('陈奕迅', '我们', 100.0), 0.94),
 (('田馥甄', '爱了很久的朋友', 100.0), 0.935)]

In [62]:
# 生成artist_id => 歌手名对应表  
singer_dict = dict()
for one in songs:
    if one['artist_id'] != 0:
        singer_dict[one['artist']] = one['artist_id']
    singer_dict[one['artist_id']]= one['artist']
print(len(singer_dict))

184991


In [52]:
# 遍历所有歌单，生成模型 singer2vec
playlist_sequence = []
for one in contents_dst:
    song_sequence = list(map(lambda x:str(song_dict[x]['artist_id']), one['songs']))
    for _ in range(len(song_sequence)//10):
        new = list(song_sequence)
        random.shuffle(new)
        playlist_sequence.append(new)

# out 为输出模型
out = 'singer2vec.model'
model = Word2Vec(sentences=playlist_sequence, size=100, window=5, min_count=5)
model.save(out)

In [60]:
# 测试模型singer2vec
def test_singer(singer_id):
    model = gensim.models.Word2Vec.load("singer2vec.model")
    result = model.most_similar(str(singer_id))
    print("与 "+singer_dict[singer_id]+" 相似的歌手是：")
    for one in result:
        print((singer_dict[int(one[0])], round(float(one[1]), 3)))

In [63]:
test_singer(singer_dict['林宥嘉'])

与 林宥嘉 相似的歌手是：
('田馥甄', 0.735)
('周兴哲', 0.716)
('陈势安', 0.684)
('郭顶', 0.672)
('李代沫', 0.652)
('赵紫骅', 0.634)
('张禄籴', 0.626)
('陈嘉桦', 0.626)
('光泽', 0.625)
('AllenRock', 0.623)


In [ ]:
with open('likes.txt', 'r', encoding='utf8') as file:
    user_test = eval(file.readline())
for i in user_test.keys():
    sort_li = collections.defaultdict()
    print('\n第{}个user喜欢的歌有：'.format(i))
    for song_id_list in user_test[i]:
        result=(song_dict[int(song_id_list)]['artist'],song_dict[int(song_id_list)]['name'])
        print(result)

In [ ]:
# 为9个user做推荐测试
with open('user_result.txt', 'w', encoding='utf8') as file:
    for i in user_test.keys():
        sort_li = collections.defaultdict(float)
        for song_id_list in user_test[i]:
            result=test(int(song_id_list))
            for song in result:
                sort_li[song[0]] += song[1] * song[0][2] 
        print('为第'+i+'个user推荐的歌曲为：')
        result = [(one, sort_li[one]) for one in sorted(sort_li, key=lambda x:sort_li[x], reverse=True)][:10]
        print(result)
        file.write('为第'+i+'个user推荐的歌曲为：'+str(result)+'\n')


In [78]:
# 热门歌曲推荐
with open('hot.txt', 'r', encoding='utf8') as file:
    hot_songs = file.readlines()
    
for one in hot_songs[4:21]:
    test(int(one.split()[0]))


与 Pianoboy高至豪 的 The truth that you leave 相似的歌曲有：
DJ Okawari Flower Dance
Nigel Silin Sakura Tears
Cymophane Tassel
Anan Ryoko Refrain
Martin Ermen River Flows In You
K. Williams 菊次郎的夏天
广东雨神 广东十年爱情故事
iris Letter
Valentin A Little Story
Maksim Mrvica Croatian Rhapsody

与 Dragon Pig 的 全部都是你 相似的歌曲有：
薛明媛 非酋
新乐尘符 123我爱你
校长 带你去旅行
白小白 最美情侣
刘佳 爱的就是你
萧全 不仅仅是喜欢
Ryan.B 再也没有
麦小兜 9420
KT 7538(Me U-Remix)
Gibb-Z 陪着你走

与 鹿先森乐队 的 春风十里 相似的歌曲有：
任素汐 我要你
陈鸿宇 理想三旬
刘昊霖 儿时
Jam 七月上
谢春花 借我
花粥 遥不可及的你
谢春花 我一定会爱上你
刘瑞琦 房间
房东的猫 秋酿
陈绮贞 我喜欢上你时的内心活动

与 Vicetone 的 Nevada 相似的歌曲有：
Alan Walker The Spectre
Alan Walker Alone
Mike Perry The Ocean (Radio Edit)
Martin Tungevaag Wicked Wonderland (Radio Edit)
Alan Walker All Falls Down
TheFatRat Monody
Alan Walker Faded
Alan Walker Hymn For The Weekend [Remix]
Aash Mehta Clsr (Aash Mehta Flip)
Anthony Keyrouz Something Just Like This

与 谢安琪 的 钟无艳 相似的歌曲有：
郑欣宜 上心
陈柏宇 你瞒我瞒
梁汉文 七友
谢安琪 喜帖街
杨千嬅 可惜我是水瓶座
邓丽欣 电灯胆
吴雨霏 吴哥窟
周柏豪 够钟
杨千嬅 小城大事
金玟岐 岁月神偷

与 孙燕姿 的 遇见 相似的歌曲有：
陈小春 独家记忆
莫文蔚 阴天
林俊杰 修